# PreProcess SQUAD and save it in Elactic Search


We will be communicating with our Elasticsearch document store via Haystack. First, we need to install Haystack using pip:



```
pip install farm-haystack
```

We will start by indexing the SQuAD dev data. So let's load that into our notebook first.



In [32]:
import json

with open('./data/squad/dev-v2.0.json', 'r') as f:
    squad = json.load(f)

In [34]:
squad['data'][0]['paragraphs'][0]

{'qas': [{'question': 'In what country is Normandy located?',
   'id': '56ddde6b9a695914005b9628',
   'answers': [{'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159}],
   'is_impossible': False},
  {'question': 'When were the Normans in Normandy?',
   'id': '56ddde6b9a695914005b9629',
   'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
    {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
    {'text': '10th and 11th centuries', 'answer_start': 94},
    {'text': '10th and 11th centuries', 'answer_start': 94}],
   'is_impossible': False},
  {'question': 'From which countries did the Norse originate?',
   'id': '56ddde6b9a695914005b962a',
   'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
    {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
    {'text': 'Denmark, Iceland and Norway', 'answer_star

In [35]:
squad['data'][-1]['paragraphs'][0]

{'qas': [{'question': 'What concept did philosophers in antiquity use to study simple machines?',
   'id': '573735e8c3c5551400e51e71',
   'answers': [{'text': 'force', 'answer_start': 46},
    {'text': 'force', 'answer_start': 46},
    {'text': 'the concept of force', 'answer_start': 31},
    {'text': 'the concept of force', 'answer_start': 31},
    {'text': 'force', 'answer_start': 46},
    {'text': 'force', 'answer_start': 46}],
   'is_impossible': False},
  {'question': 'What was the belief that maintaining motion required force?',
   'id': '573735e8c3c5551400e51e72',
   'answers': [{'text': 'fundamental error', 'answer_start': 387},
    {'text': 'A fundamental error', 'answer_start': 385},
    {'text': 'A fundamental error', 'answer_start': 385},
    {'text': 'A fundamental error', 'answer_start': 385},
    {'text': 'A fundamental error', 'answer_start': 385},
    {'text': 'A fundamental error', 'answer_start': 385}],
   'is_impossible': False},
  {'question': 'Who had mathmatical 

We can see that both 'answers' and 'plausible_answers' fields appear in this dataset. However this time, we can see multiple answers that seem to be duplicates - so we'll need to adjust our logic to deal with that.



We'll try and produce the  format  where we have a list of dictionaries where each dictionary contains a single question, answer, and context.

In [36]:
# initialize list where we will place all of our data
new_squad = []

# we need to loop through groups -> paragraphs -> qa_pairs
for group in squad['data']:
    for paragraph in group['paragraphs']:
        # we pull out the context from here
        context = paragraph['context']
        for qa_pair in paragraph['qas']:
            # we pull out the question
            question = qa_pair['question']
            # now the logic to check if we have 'answers' or 'plausible_answers'
            if 'answers' in qa_pair.keys() and len(qa_pair['answers']) > 0:
                answer_list = qa_pair['answers']
            elif 'plausible_answers' in qa_pair.keys() and len(qa_pair['plausible_answers']) > 0:
                answer_list = qa_pair['plausible_answers']
            else:
                # this shouldn't happen, but just in case we just set answer = []
                answer_list = []
            # we want to pull our the 'text' of each answer in our list of answers
            answer_list = [item['text'] for item in answer_list]
            # we can remove duplicate answers by converting our list to a set, and then back to a list
            answer_list = list(set(answer_list))
            # we iterate through each unique answer in the answer_list
            for answer in answer_list:
                # append dictionary sample to parsed squad
                new_squad.append({
                    'question': question,
                    'answer': answer,
                    'context': context
                })

In [37]:
new_squad[-2:]

[{'question': 'What force leads to a commonly used unit of mass?',
  'answer': 'kilogram-force',
  'context': 'The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kgf) (sometimes kilopond), is the force exerted by standard gravity on one kilogram of mass. The kilogram-force leads to an alternate, but rarely used unit of mass: the metric slug (sometimes mug or hyl) is that mass that accelerates at 1 m·s−2 when subjected to a force of 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.'},
 {'question': 'What force is part of the modern SI system?',
  'answer': 'kilogram-force',
  'context': 'The pound-force has a metric co

In [38]:
import os
with open(os.path.join("./data/squad/", 'dev.json'), 'w') as f:
    json.dump(new_squad, f)

In [39]:
import json

with open('./data/squad/dev.json', 'r') as f:
    squad = json.load(f)

Next, we initialize a connection between Haystack and our local Elasticsearch instance like so:

In [40]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Great, we've established our connection, now let's try querying our Elasticsearch instance. We will do this through the `requests` library.

In [41]:
import requests

Let's check our cluster *health* (eg the general status of our Elasticsearch instance). We do this by sending a **GET** request to the `_cluster/health` endpoint.

In [42]:
res = requests.get('http://localhost:9200/_cluster/health')

res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 6,
 'active_shards': 6,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 3,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 66.66666666666666}

Okay we can see that the cluster is definitely running. The cluster status is *yellow*, ideally we want to aim for *green* but the reason we see yellow here is because not all replica shards have been allocated to nodes. 
## Adding Data

Right now our Elasticsearch instance contains a single, empty index called *'squad_docs'*. We need to populate this with our `squad` data. We populate our index through the `document_store.write_documents(<input_data>)` method, where our *\<input_data\>* must be a list of dictionaries in the format:

```json
{
    'content': '<document text here>',
    'meta': {
        'other': '<other info here>'
    }
}
```

We **must** include the `'content'` key. The *content* must contain the text from each sample, which in our case is a *context* string. The `'meta'` data is optional, but is usually used to contain anything else that might be relevant, so for example we might want to include the *group* that the context came from (eg 'Beyonce', or 'Matter').

In [44]:
squad_docs = []

for sample in squad:
    squad_docs.append({
        'content': sample['context']
    })

In [43]:
len(squad_docs)

16209

Then we add our data to the index like this:

In [45]:
document_store.write_documents(squad_docs)

In [46]:
len(squad_docs)

16209

In [47]:
requests.get('http://localhost:9200/squad_docs/_count').json()

{'count': 1204,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [48]:
results = document_store.get_all_documents()

In [49]:
len(results)
print(results[-1])

<Document: id=3d2dc0db304889bea0275affd31585ee, content='The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kg...'>


When we're retrieving data from Elasticsearch we will be retrieving documents using either the TF-IDF, or BM25 algorithms.

**TF-IDF** is a common *relevance* scoring algorithm, the built is calculated using:

* **TF**, the volume of words in the query (question) that appear in the document.

* **IDF**, the inverse of the fraction of documents that contain the same word (eg common words like *'the'* don't score well, whereas *'Beyonce'* would).

We integrate TD-IDF using:

In [50]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store)

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


We can see here that when building our retriever, it identified a total of *16209* 'candidate paragraphs'. These are all of the contexts from our `squad` data:

In [51]:
len(squad)

16209

For now, we can return data from Elasticsearch, using the **TF-IDF** algorithm, with the `retrieve` method.

In [52]:
query = "Physics is a very abstract subject"

retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'With modern insights into quantum mechanics and technology that can accelerate particles close to the speed of light, particle physics has devised a Standard Model to describe forces between particles smaller than atoms. The Standard Model predicts that exchanged particles called gauge bosons are the fundamental means by which forces are emitted and absorbed. Only four main interactions are known: in order of decreasing strength, they are: strong, electromagnetic, weak, and gravit

This query returns a huge number of duplicates. The reason we have these is because our data contained duplicates of the same context because each context could be tied to several different questions. So now, we need to restart by first deleting everything inside our *squad_docs* index. Then re-indexing our deduplicated data.

We can delete every document in our index by sending a **POST** request to the `<index_name>/_delete_by_query` endpoint:

In [53]:
res = requests.post('http://localhost:9200/squad_docs/_delete_by_query',
                    json={
                        'query': {
                            'match_all': {}
                        }
                    })

res.json()

{'took': 450,
 'timed_out': False,
 'total': 1204,
 'deleted': 1204,
 'batches': 2,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

Our response shows `'deleted': 16209`, which means all *16209* documents have been deleted from our *squad_docs* index. We can confirm this by calling the `<index_name>/_count` endpoint too:

In [54]:
res = requests.get('http://localhost:9200/squad_docs/_count')

res.json()

{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

Now that we've cleared the index, it's time to remove duplicates from our SQuAD contexts and re-index them.

In [55]:
# create list of contexts (we cannot do this using current dictionary format)
contexts = [sample['context'] for sample in squad]

# convert to set to remove duplicates, then back to list
contexts = list(set(contexts))

# convert back to dictionary format we need
squad_docs = [{'content': sample} for sample in contexts]

In [56]:
len(squad_docs)

1204

In [67]:
squad_docs[-1]


{'content': "Moderate and reformist Islamists who accept and work within the democratic process include parties like the Tunisian Ennahda Movement. Jamaat-e-Islami of Pakistan is basically a socio-political and democratic Vanguard party but has also gained political influence through military coup d'état in past. The Islamist groups like Hezbollah in Lebanon and Hamas in Palestine participate in democratic and political process as well as armed attacks, seeking to abolish the state of Israel. Radical Islamist organizations like al-Qaeda and the Egyptian Islamic Jihad, and groups such as the Taliban, entirely reject democracy, often declaring as kuffar those Muslims who support it (see takfirism), as well as calling for violent/offensive jihad or urging and conducting attacks on a religious basis."}

Finally, we can re-index our Elasticsearch as we did before.

In [57]:
document_store.write_documents(squad_docs)

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [58]:
res = requests.get('http://localhost:9200/squad_docs/_count')

res.json()

{'count': 1204,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

Because we have changed the contents of our index, we initialize our retriever once more.

In [59]:
retriever = TfidfRetriever(document_store)

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


And this time we see that our retriever found *1204* documents (much less than the *16209* we found before). Now it's time to query our data again!

In [62]:
retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': 0.7329360380503047, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'A computational problem can be viewed as an infinite collection of instances together with a solution for every instance. The input string for a computational problem is referred to as a problem instance, and should not be confused with the problem itself. In computational complexity theory, a problem refers to the abstract question to be solved. In contrast, an instance of this problem is a rather concrete utterance, which can serve as the input for a decision probl

In [65]:
query="Who did Emma Marry?"
retriever.retrieve(query)

[<Document: {'content': "The Normans were in contact with England from an early date. Not only were their original Viking brethren still ravaging the English coasts, they occupied most of the important ports opposite England across the English Channel. This relationship eventually produced closer ties of blood through the marriage of Emma, sister of Duke Richard II of Normandy, and King Ethelred II of England. Because of this, Ethelred fled to Normandy in 1013, when he was forced from his kingdom by Sweyn Forkbeard. His stay in Normandy (until 1016) influenced him and his sons by Emma, who stayed in Normandy after Cnut the Great's conquest of the isle.", 'content_type': 'text', 'score': 0.8395045765246445, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '9b09ebf4eb65bd2133a6374e1f5ef5a'}>,
 <Document: {'content': "These attacks resonated with conservative Muslims and the problem did not go away with Saddam's defeat either, since American troops remained stationed in t

Now we're returning a set of relevant documents, without duplicates.

Finally, let's return back to the other *sparse retriever* that we can use with Elasticsearch. We already used **TF-IDF**, by switching `TfidfRetriever` for `BM25Retriever` we can switch to the **BM25** algorithm, which is an *improved* version of **TF-IDF** and is recommended by Haystack.

So, let's initialize that and make another query with it.

In [66]:
# import BM25 retriever
from haystack.nodes import BM25Retriever

# intialize
retriever = BM25Retriever(document_store)

# and query
retriever.retrieve(query)

[<Document: {'content': "The Normans were in contact with England from an early date. Not only were their original Viking brethren still ravaging the English coasts, they occupied most of the important ports opposite England across the English Channel. This relationship eventually produced closer ties of blood through the marriage of Emma, sister of Duke Richard II of Normandy, and King Ethelred II of England. Because of this, Ethelred fled to Normandy in 1013, when he was forced from his kingdom by Sweyn Forkbeard. His stay in Normandy (until 1016) influenced him and his sons by Emma, who stayed in Normandy after Cnut the Great's conquest of the isle.", 'content_type': 'text', 'score': 0.8210750391274714, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '9b09ebf4eb65bd2133a6374e1f5ef5a'}>,
 <Document: {'content': "Some elements of the Brotherhood, though perhaps against orders, did engage in violence against the government, and its founder Al-Banna was assassinated in